## References

In [1]:
# https://radimrehurek.com/gensim/tut1.html
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
# https://docs.python.org/2/library/re.html

## Notebook Setup

In [3]:
import numpy as np

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\Users\s.birk\Anaconda3\envs\esade-topic-analysis-twitter\python.exe"
  * The NumPy version is: "1.20.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: The specified module could not be found.


In [2]:
# Import libraries
import io
import logging
import nltk
import pandas as pd
import pickle
from gensim import corpora
from gensim.models import Phrases
from nltk.stem.wordnet import WordNetLemmatizer

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\Users\s.birk\Anaconda3\envs\esade-topic-analysis-twitter\python.exe"
  * The NumPy version is: "1.20.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: The specified module could not be found.


In [ ]:
# Download stopwords and lemmatizer from nltk package
nltk.download("stopwords")
nltk.download("wordnet")

In [ ]:
# log events
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Load and Inspect Dataset

In [ ]:
# read data with timestamp as index
tweets = pd.read_csv("tweets.csv", encoding="latin1", parse_dates=True, 
                     index_col="created", usecols=range(1,28))

In [ ]:
# inspect dataframe
tweets.head()

In [ ]:
# display dataframe info
tweets.info()

In [ ]:
# describe dataframe 
tweets.describe()

In [ ]:
# divide dataset according to language: extract english language
english_tweets = tweets[tweets["language3"] == "ENGLISH"].copy()

## Data Cleaning and Preparation

### Preprocess Twitter Text

In [ ]:
# display text
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

english_tweets["text"]

In [ ]:
# remove links
english_tweets["text_clean"] = english_tweets["text"].str.replace(r"http\S+", "")
# remove emoticons
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"<.*>", "")
# remove punctuation, special characters etc.
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"&amp", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\.", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\,", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\;", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\-", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\"", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace("\\\\", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace("\/", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace("\*", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"@", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\n", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\|", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"W//", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"!", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"~", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r")", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"(", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"?", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r":", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\{", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"\}", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"_", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"h:[0-9]+m:[0-9]+s", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[0-9]+", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"w/", " ")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x97]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x96]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x95]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x94]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x93]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x92]+", "'")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x91]+", "")
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r"[\x85]+", "")

# reduce white spaces to 1
english_tweets["text_clean"] = english_tweets["text_clean"].str.replace(r" +", " ")

In [ ]:
# display cleaned text
english_tweets["text_clean"]

In [ ]:
# reorder columns
english_tweets.columns
cols = ['text', 'text_clean', 'favoriteCount', 'replyToSN', 'truncated', 'replyToSID',
       'replyToUID', 'statusSource', 'retweetCount', 'longitude', 'latitude',
       'id_seccion', 'horaPeticion', 'id_distrito', 'grupoHora',
       'id_seccion_xy', 'favoriteCountOutlier', 'retweetCountOutlier',
       'tweetcount', 'movement', 'language3', 'dayofweek', 'weeknumber',
       'month', 'idBarrio_xy', 'idBarrio', 'user']

english_tweets = english_tweets[cols]

#### In order to train the topic models, 3 different pooling methods for the creation of documents are used: No Pooling (1), User Pooling (2) and Hashtag Pooling (3).

### Training Documents Option 1 (No Pooling)

In [ ]:
# treat every tweet as a different document (no pooling)
documents = english_tweets["text_clean"].tolist()

### Training Documents Option 2 (User Pooling)

In [ ]:
# treat all tweets by one user as one single document (user pooling)
user_combined = english_tweets[["text_clean","user"]].groupby("user")["text_clean"].apply(lambda x: "".join(x))
documents_user_pooling = user_combined.tolist()

### Training Documents Option 3 (Hashtag Pooling)

In [ ]:
# treat all tweets with the same hashtag as one single document (hashtag pooling)

# find all hashtags
english_tweets["hashtags"] = english_tweets["text_clean"].str.findall(r'#.*?(?=\s|$)')

# separate hashtags in columns
hashtags_tweets = pd.DataFrame(english_tweets["hashtags"].tolist(),
                               columns=["hashtag1", "hashtag2", "hashtag3", "hashtag4",
                                        "hashtag5", "hashtag6", "hashtag7", "hashtag8",
                                        "hashtag9", "hashtag10", "hashtag11", "hashtag12",
                                        "hashtag13"])

# join hashtags with tweet text
hashtags_tweets.index = english_tweets.index
hashtags_tweets = english_tweets.join(hashtags_tweets)

In [ ]:
# create one dataframe with text for each hashtag column and save them in a dictionary
dict = {}
for index, item in enumerate(["hash1", "hash2", "hash3", "hash4", "hash5",
                              "hash6", "hash7", "hash8", "hash9", "hash10",
                              "hash11", "hash12", "hash13"]):
    dict[item] = hashtags_tweets[["hashtag" + str(index + 1), "text_clean"]].copy()
    dict[item].columns = ["hashtag", "text"]
    dict[item].dropna(inplace=True)

# concatenate all dataframes to one dataframe (the result is a dataframe
# where there is text for each hashtag found)
hashtags = pd.DataFrame()
for item in dict:
    hashtags = pd.concat([hashtags, dict[item]])

# combine text for each hashtag
hashtags_combined = hashtags.groupby("hashtag")["text"].apply(lambda x: "".join(x))

In [ ]:
# remove some generic hashtags that cover a lot of different topics
hashtags_combined.drop(["#Barcelona", "#Catalunya", "#Spain", "#BCN", "#BARCELONA",
                        "#Espana", "#BarcelonaSpain"], inplace=True)

In [ ]:
# create documents
documents_hashtag_pooling = hashtags_combined.tolist()

### Analyze Documents

In [ ]:
# calculate number of terms per document
total = []
for item in documents:
    total.append(len(item.split()))

In [ ]:
# calculate total number of terms
sum = 0
for number in total:
    sum += number

In [ ]:
# calculate average number of terms per document
sum/len(documents)

### Prepare Test Documents

#### The trained topic models will then be used to determine the topics of test documents.

#### The first objective of the research is to analyze the distribution of topics over the districts. For this purpose, district pooling is used to create the documents that will be tested.

In [ ]:
# merge all tweets from each district (district pooling) and treat them as one single document respectively
district_combined = english_tweets[["text_clean","id_distrito"]].groupby("id_distrito")["text_clean"].apply(lambda x: "".join(x))
documents_district_pooling = district_combined.tolist()

In [ ]:
# check documents
district_combined

In [ ]:
# check if order is right
district_combined.index # correct

#### The second objective is to look at the dynamic topic development over time. For this purpose, the dataset is divided according to time and documents are created on this basis.

#### Divide dataframe according to month

In [ ]:
# sort index
sorted_tweets = english_tweets.sort_index()

In [ ]:
# check first and last date
print(sorted_tweets.index[0]) # June 2017
print(sorted_tweets.index[-1]) # December 2017 (very incomplete)

In [ ]:
# create column that contains the month of the tweets
sorted_tweets['month'] = sorted_tweets.index.month

In [ ]:
# split dataframe according to month
june = sorted_tweets.loc['2017-06-01':'2017-06-30']
july = sorted_tweets.loc['2017-07-01':'2017-07-31']
august = sorted_tweets.loc['2017-08-01':'2017-08-31']
september = sorted_tweets.loc['2017-09-01':'2017-09-30']
october = sorted_tweets.loc['2017-10-01':'2017-10-31']
november = sorted_tweets.loc['2017-11-01':'2017-11-30']
december = sorted_tweets.loc['2017-12-01':'2017-12-31']

In [ ]:
# count number of tweets per month
len(june)

In [ ]:
# merge all tweets from each month and treat them as one document respectively
months_combined = english_tweets[["text_clean","month"]].groupby("month")["text_clean"].apply(lambda x: "".join(x))
documents_month_pooling = months_combined.tolist()

In [ ]:
# check how many documents
len(documents_month_pooling) # should be 7

In [ ]:
# check order of documents
months_combined.index # Aug, Dec, Jul, Jun, Nov, Oct, Sep

In [ ]:
# count number of characters
len(documents_month_pooling[6])

In [ ]:
# convert strings to wordlist
aug_words = documents_month_pooling[0].split()
dec_words = documents_month_pooling[1].split()
jul_words = documents_month_pooling[2].split()
jun_words = documents_month_pooling[3].split()
nov_words = documents_month_pooling[4].split()
oct_words = documents_month_pooling[5].split()
sept_words = documents_month_pooling[6].split()

In [ ]:
# count number of words per month
print(len(jun_words))
print(len(jul_words))
print(len(aug_words))
print(len(sept_words))
print(len(oct_words))
print(len(nov_words))
print(len(dec_words))

In [ ]:
# count unique words
print(len(set(jun_words)))
print(len(set(jul_words)))
print(len(set(aug_words)))
print(len(set(sept_words)))
print(len(set(oct_words)))
print(len(set(nov_words)))
print(len(set(dec_words)))

In [ ]:
# merge all tweets from one district and one month and treat them as one document respectively
districts_per_month_combined = sorted_tweets[["text_clean","month","id_distrito"]].groupby(["month","id_distrito"])["text_clean"].apply(lambda x: "".join(x))
documents_district_per_month_pooling = districts_per_month_combined.tolist()

In [ ]:
# check order of documents
districts_per_month_combined.index 

In [ ]:
# check how many documents
len(documents_district_per_month_pooling)

In [ ]:
# inspect dataframe
districts_per_month_combined

### Preprocess Documents for NMF Topic Modeling Method

In [ ]:
# create copy of no pooling documents
nmf_documents = list(documents)

In [ ]:
# transform to lower case
for doc_idx, doc in enumerate(nmf_documents):
    nmf_documents[doc_idx] = nmf_documents[doc_idx].lower()

In [ ]:
# delete stopwords
for doc_idx, doc in enumerate(nmf_documents):
    nmf_documents[doc_idx] = doc.replace(" year ", " ").replace(" to ", " ").replace(" on ", " ").replace(" wa ", " ").replace(" #yourup ", " ").replace(" de ", " ").replace(" just ", " ").replace(" posted ", " ").replace(" photo ", " ").replace(" la ", " ").replace(" del ", " ").replace(" en ", " ").replace(" los ", " ").replace(" el ", " ").replace(" las ", " ").replace(" barcelona ", " ").replace(" #bcn ", " ").replace(" just ", " ").replace(" cada ", " ").replace(" nuestra ", " ").replace(" around ", " ").replace(" spanish ", " ").replace(" día ", " ").replace(" dia ", " ").replace(" #photo ", " ").replace(" first ", " ").replace(" thing ", " ").replace(" last ", " ").replace(" #spain ", " ").replace(" carrer ", " ").replace(" make ", " ").replace(" &lt ", " ").replace(" &gt ", " ").replace(" de ", " ").replace(" for ", " ").replace(" a ", " ").replace(" of ", " ").replace(" the ", " ").replace(" and ", " ").replace(" to ", " ").replace(" in ", " ").replace(" at ", " ").replace(" by ", " ").replace(" one ", " ").replace(" day ", " ").replace(" get ", " ").replace(" españa ", " ").replace(" #españa ", " ").replace(" #repost ", " ").replace(" since ", " ").replace(" still ", " ").replace(" never ", " ").replace(" thank ", " ").replace(" two ", " ").replace(" think ", " ").replace(" could ", " ").replace(" many ", " ").replace(" even ", " ").replace(" the ", " ").replace(" igers ", " ").replace(" que ", " ").replace(" many ", " ").replace(" con ", " ").replace(" un ", " ").replace(" wa ", " ").replace(" bcn ", " ").replace(" d'horta ", " ").replace(" ever ", " ").replace(" come ", " ").replace(" #ig ", " ").replace(" el ", " ").replace(" i'm ", " ").replace(" i've ", " ").replace(" always ", " ").replace(" le ", " ").replace(" what's ", " ").replace(" #barcelone ", " ").replace(" like ", " ").replace(" last ", " ").replace(" back ", " ").replace(" thanks ", " ").replace(" #barna ", " ").replace(" spain ", " ").replace(" yo ", " ").replace(" #yo ", " ").replace(" el ", " ").replace(" #el ", " ").replace(" barcelona ", " ").replace(" #barcelona ", " ")    

In [ ]:
# display preprocessed documents
nmf_documents

### Save Training and Test Documents

In [ ]:
with io.open('documents.txt', 'w', encoding='utf-8') as f:
    for item in documents:
        f.write(item + "\n")
with io.open('documents_user_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_user_pooling:
        f.write(item + "\n")
with io.open('documents_hashtag_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_hashtag_pooling:
        f.write(item + "\n")
with io.open('documents_district_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_district_pooling:
        f.write(item + "\n")
with io.open('documents_month_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_month_pooling:
        f.write(item + "\n")
with io.open('documents_district_per_month_pooling.txt', 'w', encoding='utf-8') as f:
    for item in documents_district_per_month_pooling:
        f.write(item + "\n")
        
with open('documents_no_pooling.p', 'wb') as fp:
    pickle.dump(documents, fp)
    
with open('documents_user_pooling.p', 'wb') as fp:
    pickle.dump(documents_user_pooling, fp)
    
with open('documents_hashtag_pooling.p', 'wb') as fp:
    pickle.dump(documents_hashtag_pooling, fp)
    
with open('nmf_documents_no_pooling.p', 'wb') as fp:
    pickle.dump(nmf_documents, fp)

In [ ]:
nmf_documents

### Tokenize Training Documents

In [ ]:
# we can simply tokenize by space thanks to the previous preprocessing
texts_no_pooling = [[word for word in document.lower().split()]
          for document in documents]

texts_user_pooling = [[word for word in document.lower().split()]
          for document in documents_user_pooling]

texts_hashtag_pooling = [[word for word in document.lower().split()]
          for document in documents_hashtag_pooling]

### Save Unpreprocessed Tokenized Training Documents

In [ ]:
with open('tokenized_documents_no_pooling_unpp.p', 'wb') as fp:
    pickle.dump(texts_no_pooling, fp)
    
with open('tokenized_documents_user_pooling_unpp.p', 'wb') as fp:
    pickle.dump(texts_user_pooling, fp)
    
with open('tokenized_documents_hashtag_pooling_unpp.p', 'wb') as fp:
    pickle.dump(texts_hashtag_pooling, fp)

### Further Preprocessing of Training Documents after Tokenization

In [ ]:
# remove numbers, but not words that contain numbers.
texts_no_pooling = [[token for token in doc if not token.isnumeric()] for doc in texts_no_pooling]
texts_user_pooling = [[token for token in doc if not token.isnumeric()] for doc in texts_user_pooling]
texts_hashtag_pooling = [[token for token in doc if not token.isnumeric()] for doc in texts_hashtag_pooling]

In [ ]:
# remove words that are only one character.
texts_no_pooling = [[token for token in doc if len(token) > 1] for doc in texts_no_pooling]
texts_user_pooling = [[token for token in doc if len(token) > 1] for doc in texts_user_pooling]
texts_hashtag_pooling = [[token for token in doc if len(token) > 1] for doc in texts_hashtag_pooling]

In [ ]:
# lemmatize all words in all documents.
lemmatizer = WordNetLemmatizer()
texts_no_pooling = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts_no_pooling]
texts_user_pooling = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts_user_pooling]
texts_hashtag_pooling = [[lemmatizer.lemmatize(token) for token in doc] for doc in texts_hashtag_pooling]

In [ ]:
# # ignore this part! computing bigrams did not improve models but made them worse!

# # compute bigrams
# # add bigrams and trigrams to docs (only ones that appear 5 times or more)
# bigram = Phrases(texts_no_pooling, min_count=10)
# for idx in range(len(texts_no_pooling)):
#     for token in bigram[texts_no_pooling[idx]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             texts_no_pooling[idx].append(token)
#             
# bigram = Phrases(texts_user_pooling, min_count=10)
# for idx in range(len(texts_user_pooling)):
#     for token in bigram[texts_user_pooling[idx]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             texts_user_pooling[idx].append(token)
#             
# bigram = Phrases(texts_hashtag_pooling, min_count=10)
# for idx in range(len(texts_hashtag_pooling)):
#     for token in bigram[texts_hashtag_pooling[idx]]:
#         if '_' in token:
#             # Token is a bigram, add to document.
#             texts_hashtag_pooling[idx].append(token)

## Save Tokenized Training Documents

In [ ]:
with open('tokenized_documents_no_pooling.p', 'wb') as fp:
    pickle.dump(texts_no_pooling, fp)
    
with open('tokenized_documents_user_pooling.p', 'wb') as fp:
    pickle.dump(texts_user_pooling, fp)
    
with open('tokenized_documents_hashtag_pooling.p', 'wb') as fp:
    pickle.dump(texts_hashtag_pooling, fp)

## Refine and Vectorize Corpora

In [ ]:
# define function to refine and vectorize corpus 
# (remove stopwords, very frequent and very infrequent words etc.)

# define stopwords
stpwords = "for a of the and to in at by one #yo #el day get españa #yourup #españa #repost yo el since still never thank two think could many even ha igers th que con un wa bcn d'horta ever come #ig el i'm i've always le what's #barcelone like last back thanks #barna spain barcelona #barcelona cada nuestra around spanish día dia #photo first thing last #spain carrer make &lt &gt de la del en las barcelona #bcn just posted photo year wa".split()

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, 
                additional_stopwords=set(stpwords),
                no_below=2, no_above=0.5,
                dictionary_name='tourism.dict', corpus_name='tourism.mm'):
    print('Building dictionary...')
    dictionary = corpora.Dictionary(docs)
    stopwords = nltk_stopwords().union(additional_stopwords)
    stopword_ids = map(dictionary.token2id.get, stopwords)
    dictionary.filter_tokens(stopword_ids)
    dictionary.compactify()
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
    dictionary.compactify()
    dictionary.save(dictionary_name)  # store the dictionary, for future reference
    
    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    corpora.MmCorpus.serialize(corpus_name, corpus)  # store to disk, for later use
    
    return (corpus, dictionary)

In [ ]:
# run function to vectorize corpora
corpus_no_pooling = prep_corpus(texts_no_pooling,
                                dictionary_name="tourism_no_pooling.dict",
                                corpus_name="tourism_no_pooling.mm")[0]
dictionary_no_pooling = prep_corpus(texts_no_pooling,
                                    dictionary_name="tourism_no_pooling.dict",
                                    corpus_name="tourism_no_pooling.mm")[1]

corpus_user_pooling = prep_corpus(texts_user_pooling,
                                    dictionary_name="tourism_user_pooling.dict",
                                    corpus_name="tourism_user_pooling.mm")[0]
dictionary_user_pooling = prep_corpus(texts_user_pooling,
                                    dictionary_name="tourism_user_pooling.dict",
                                    corpus_name="tourism_user_pooling.mm")[1]

corpus_hashtag_pooling = prep_corpus(texts_hashtag_pooling,
                                    dictionary_name="tourism_hashtag_pooling.dict",
                                    corpus_name="tourism_hashtag_pooling.mm")[0]
dictionary_hashtag_pooling = prep_corpus(texts_hashtag_pooling,
                                    dictionary_name="tourism_hashtag_pooling.dict",
                                    corpus_name="tourism_hashtag_pooling.mm")[1]

## Apply Function to Preprocess Test Documents (Before Testing Them with Topic Models)

#### This function has to include all the same steps that were applied to the training documents!

In [ ]:
# define function
def preprocess(docs):
    ''' Conduct all preprocessing steps that are conducted to train the LDA model'''
    
    # tokenize documents
    tokenized = [[word for word in document.lower().split()]
          for document in docs]
    
    # remove words that are only one character
    tokenized = [[token for token in doc if len(token) > 1] for doc in tokenized]
    
    # lemmatize all words
    lemmatizer = WordNetLemmatizer()
    lemmatized = [[lemmatizer.lemmatize(token) for token in doc] for doc in tokenized]
    
    # define stopwords
    stpwords = "for a of the and to in at by one #yo #el day get #yourup españa #españa #repost yo el since still never thank two think could many even ha igers th que con un wa bcn d'horta ever come #ig el i'm i've always le what's #barcelone like last back thanks #barna spain barcelona #barcelona cada nuestra around spanish día dia #photo first thing last #spain carrer make &lt &gt de la del en las barcelona #bcn just posted photo year wa".split()
    
    # get stopwords from nltk
    def nltk_stopwords():
        return set(nltk.corpus.stopwords.words('english'))

    # combine stopwords
    stopwords = nltk_stopwords().union(stpwords)
    
    # remove stopwords
    preprocessed = [[token for token in document if token not in stopwords] for document in lemmatized]
    
    return preprocessed

In [ ]:
# apply function to test documents
texts_district_pooling = preprocess(documents_district_pooling)
texts_month_pooling = preprocess(documents_month_pooling)
texts_district_per_month_pooling = preprocess(documents_district_per_month_pooling)

## Save Preprocessed Test Documents

In [ ]:
with open('tokenized_documents_district_pooling.p', 'wb') as fp:
    pickle.dump(texts_district_pooling, fp)
    
with open('tokenized_documents_month_pooling.p', 'wb') as fp:
    pickle.dump(texts_month_pooling, fp)
    
with open('tokenized_documents_district_per_month_pooling.p', 'wb') as fp:
    pickle.dump(texts_district_per_month_pooling, fp)

In [ ]:
# # ignore this part! just example code!

# # map tokens to ids
# print(dictionary_no_pooling.token2id)
# print(dictionary_user_pooling.token2id)
# print(dictionary_hashtag_pooling.token2id)

In [ ]:
# # ignore this part! just example code!

# # convert new document to vector 
# new_doc = "Sagrada Familia is amazing"
# new_vec_no_pooling = dictionary_no_pooling.doc2bow(new_doc.lower().split())
# print(new_vec_no_pooling)

In [ ]:
# # ignore this part! not needed for dataset!

# # corpus streaming: one document at a time
# class MyCorpus(object):
#     def __iter__(self):
#         for line in open("corpus_no_pooling.txt"):
#             # assume there's one document per line, tokens separated by whitespace
#             yield dictionary.doc2bow(line.lower().split())
#             
# corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
# print(corpus_memory_friendly)
# 
# for vector in corpus_memory_friendly:  # load one vector into memory at a time
#     print(vector)